CONFIGURATION

In [1]:
#Installation des dépendances en amont si non-présentes dans l'environnement
!pip install numpy pandas scikit-learn

In [2]:
#Importation des librairies communes (peu prendre du temps)
import os
import sys
import numpy
import pandas as pd

In [133]:
#Chemin menant aux données
data_path = os.path.join(os.getcwd(), "source")
#création du dictionnaire
data_dict = dict()

In [134]:
# Boucle pour récupérer tous les fichiers CSV dans le dossier et les ajouter au dictionnaire
for file in os.listdir(data_path):
    if file.endswith(".csv"):
        file_path = os.path.join(data_path, file)
        dict_key = file.split('.')[0]  # Nom du fichier sans l'extension .csv
        data_dict[dict_key] = pd.read_csv(file_path)

data_dict.keys()

dict_keys(['appearances', 'club_games', 'clubs', 'competitions', 'game_events', 'game_lineups', 'games', 'player_valuations', 'players'])

FONCTIONS UTILES

In [135]:
def games_of_the_season(df_games, club_id, saison, lieu='tous'):
    """
    Récupère tous les matchs d'une saison pour un club spécifié, ou les 10 derniers matchs si moins de 5 matchs dans la saison.
    
    Paramètres:
    df_games (DataFrame) : DataFrame des matchs.
    club_id (int) : ID du club.
    saison (str) : Saison à filtrer.
    lieu (str) : 'domicile', 'exterieur', ou 'tous' pour filtrer les matchs.
    
    Retourne:
    DataFrame : Les matchs demandés avec les informations détaillées.
    """
    # Filtrage initial par saison et lieu
    if lieu == 'domicile':
        matchs_du_club = df_games[(df_games['home_club_id'] == club_id) & (df_games['season'] == saison)]
    elif lieu == 'exterieur':
        matchs_du_club = df_games[(df_games['away_club_id'] == club_id) & (df_games['season'] == saison)]
    else:
        matchs_du_club = df_games[((df_games['home_club_id'] == club_id) | (df_games['away_club_id'] == club_id)) & (df_games['season'] == saison)]
    
    # Vérification du nombre de matchs
    if len(matchs_du_club) < 5:
        # Prendre les 10 derniers matchs toutes saisons confondues si moins de 5 matchs dans la saison
        matchs_du_club = df_games[(df_games['home_club_id'] == club_id) | (df_games['away_club_id'] == club_id)]
        matchs_du_club = matchs_du_club.sort_values(by='date', ascending=False).head(10)
    else:
        # Trier par date
        matchs_du_club = matchs_du_club.sort_values(by='date', ascending=False)

    # Ajout des informations supplémentaires
    matchs_du_club['adversaire_id'] = matchs_du_club.apply(lambda row: row['away_club_id'] if row['home_club_id'] == club_id else row['home_club_id'], axis=1)
    matchs_du_club['lieu'] = matchs_du_club.apply(lambda row: 'domicile' if row['home_club_id'] == club_id else 'exterieur', axis=1)
    matchs_du_club['résultat'] = matchs_du_club.apply(lambda row: 'victoire' if (row['home_club_id'] == club_id and row['home_club_goals'] > row['away_club_goals']) or (row['away_club_id'] == club_id and row['away_club_goals'] > row['home_club_goals']) else 'défaite' if (row['home_club_id'] == club_id and row['home_club_goals'] < row['away_club_goals']) or (row['away_club_id'] == club_id and row['away_club_goals'] < row['home_club_goals']) else 'nul', axis=1)

    return matchs_du_club[['date', 'season', 'home_club_id', 'away_club_id', 'adversaire_id', 'lieu', 'home_club_goals', 'away_club_goals', 'résultat']]

# Exemple d'utilisation
saison_specifique = '2023'  # Remplacez par la saison désirée
club_id_specifique = 1041  # OLYMPIQUE LYONNAIS
matchs_de_la_saison = games_of_the_season(data_dict['games'], club_id_specifique, saison_specifique, 'tous')
print(matchs_de_la_saison)


             date  season  home_club_id  away_club_id  adversaire_id  \
48004  2023-12-20    2023          1041           995            995   
63073  2023-12-15    2023           162          1041            162   
23752  2023-12-10    2023          1041           415            415   
61247  2023-12-06    2023           244          1041            244   
63029  2023-12-02    2023           826          1041            826   
10271  2023-11-26    2023          1041          1082           1082   
30976  2023-11-12    2023           273          1041            273   
23465  2023-11-05    2023          1041           347            347   
64206  2023-10-22    2023          1041          3524           3524   
55478  2023-10-08    2023          1041          1158           1158   

            lieu  home_club_goals  away_club_goals  résultat  
48004   domicile                1                0  victoire  
63073  exterieur                0                1  victoire  
23752   domicile  

In [136]:
def last_five_games(df_games, club_id, lieu='tous'):
    """
    Récupère les 5 derniers matchs d'un club spécifié en fonction du lieu du match.
    
    Paramètres:
    df_games (DataFrame) : DataFrame des matchs.
    club_id (int) : ID du club.
    lieu (str) : 'domicile', 'exterieur', ou 'tous' pour filtrer les matchs.
    
    Retourne:
    Liste : Les identifiants (game_id) des 5 derniers matchs.
    """
    if lieu == 'domicile':
        matchs_du_club = df_games[df_games['home_club_id'] == club_id]
    elif lieu == 'exterieur':
        matchs_du_club = df_games[df_games['away_club_id'] == club_id]
    else: # 'tous'
        matchs_du_club = df_games[(df_games['home_club_id'] == club_id) | (df_games['away_club_id'] == club_id)]
    
    matchs_du_club = matchs_du_club.sort_values(by='date', ascending=False).head(5)
    return matchs_du_club['game_id'].tolist()

# Exemple d'utilisation
club_id_specifique = 1041  # OLYMPIQUE LYONNAIS
derniers_game_ids = last_five_games(data_dict['games'], club_id_specifique, 'tous') # 'domicile', 'exterieur' ou 'tous'
print(derniers_game_ids)

[4094725, 4094716, 4094708, 4094660, 4094700]


In [262]:
#last game id 
def last_game_id(df_games, club_id):
    """
    Récupère l'ID du dernier match joué par un club spécifié.

    Paramètres:
    df_games (DataFrame) : DataFrame des matchs.
    club_id (int) : ID du club.

    Retourne:
    int : ID du dernier match joué par le club.
    """
    # Filtrer les matchs impliquant le club
    matchs_du_club = df_games[(df_games['home_club_id'] == club_id) | (df_games['away_club_id'] == club_id)]
    
    # Trier par date en ordre décroissant et prendre le premier match
    dernier_match = matchs_du_club.sort_values(by='date', ascending=False).iloc[0]

    return dernier_match['game_id']

# Exemple d'utilisation
club_id_specifique = 1041  # Remplacez par l'ID du club désiré
dernier_match_id = last_game_id(df_games, club_id_specifique)
print("ID du dernier match de l'équipe", club_id_specifique, ":", dernier_match_id)


ID du dernier match de l'équipe 1041 : 4094725


In [277]:
def match_sheet(df_game_lineups, match_id, club_id):
    """
    Récupère tous les player_id qui ont participé à un match spécifique pour un club donné.
    
    Paramètres:
    df_game_lineups (DataFrame) : DataFrame des compositions d'équipe.
    match_id (int) : ID du match.
    club_id (int) : ID du club.
    
    Retourne:
    Liste : Les identifiants (player_id) des joueurs qui ont participé au match.
    """
    # Filtrer pour obtenir les lignes correspondant au match_id et club_id spécifiés
    lineup_du_match = df_game_lineups[(df_game_lineups['game_id'] == match_id) & (df_game_lineups['club_id'] == club_id)]
    
    # Récupérer les player_id
    joueurs = lineup_du_match['player_id'].tolist()

    return joueurs

# Exemple d'utilisation
match_id_specifique = 4094700
club_id_specifique = 1041  # OLYMPIQUE LYONNAIS
joueurs_participes = match_sheet(data_dict['game_lineups'], match_id_specifique, club_id_specifique)
print(joueurs_participes)

[855134, 655006, 395237, 131225, 93720, 607223, 955070, 711544, 37838, 60561, 187718, 325223, 802986, 910905, 238266, 204072, 951579, 429901, 1053147, 384060]


In [345]:
def starting_lineup_match(match_id, club_id):
    """
    Récupère les player_id des joueurs qui étaient dans la composition de départ d'un match spécifique pour un club donné.
    
    Paramètres:
    df_game_lineups (DataFrame) : DataFrame des compositions d'équipe.
    match_id (int) : ID du match.
    club_id (int) : ID du club.
    
    Retourne:
    Liste : Les identifiants (player_id) des joueurs de la composition de départ.
    """
    df_game_lineups = data_dict['game_lineups']
    # Filtrer pour obtenir les lignes correspondant au match_id, club_id et type 'starting_lineup'
    lineup_du_match = df_game_lineups[(df_game_lineups['game_id'] == match_id) & 
                                      (df_game_lineups['club_id'] == club_id) & 
                                      (df_game_lineups['type'] == 'starting_lineup')]
    
    # Récupérer les player_id
    joueurs = lineup_du_match['player_id'].tolist()

    return joueurs

# Exemple d'utilisation
match_id_specifique = 4094700  # Remplacez par l'ID du match désiré
club_id_specifique = 1041  # OLYMPIQUE LYONNAIS
joueurs_titulaires = starting_lineup_match(match_id_specifique, club_id_specifique)
print(joueurs_titulaires)

[855134, 655006, 395237, 131225, 93720, 607223, 955070, 711544, 37838, 60561, 187718]


In [346]:
def market_value_player(df_player, player_id):
    """
    Récupère la valeur marchande d'un joueur spécifié.
    
    Paramètres:
    df_player_valuations (DataFrame) : DataFrame des évaluations des joueurs.
    player_id (int) : ID du joueur.
    
    Retourne:
    float : La valeur marchande du joueur en euros.
    """
    # Trouver l'entrée correspondant au player_id
    valeur_joueur = df_player[df_player['player_id'] == player_id]['market_value_in_eur']

    # Retourner la valeur marchande, ou None si le joueur n'est pas trouvé
    return valeur_joueur.iloc[0] if not valeur_joueur.empty else None

# Exemple d'utilisation
player_id_specifique = 955070  # Remplacez par l'ID du joueur désiré
valeur_marchande = market_value_player(data_dict['player_valuations'], player_id_specifique)
print(valeur_marchande)

150000


In [347]:
def mean_team_value(df_player, liste_player_id):
    """
    Calcule la valeur marchande moyenne d'une équipe.
    
    Paramètres:
    df_player_valuations (DataFrame) : DataFrame des évaluations des joueurs.
    liste_player_id (List[int]) : Liste des ID des joueurs.
    
    Retourne:
    float : Valeur marchande moyenne de l'équipe.
    """
    valeurs = [market_value_player(df_player, player_id) for player_id in liste_player_id]
    valeurs = [v for v in valeurs if v is not None]  # Exclure les valeurs None
    return sum(valeurs) / len(valeurs) if valeurs else 0

def sum_team_value(df_player, liste_player_id):
    """
    Calcule la somme des valeurs marchandes d'une équipe.
    
    Paramètres:
    df_player_valuations (DataFrame) : DataFrame des évaluations des joueurs.
    liste_player_id (List[int]) : Liste des ID des joueurs.
    
    Retourne:
    float : Somme des valeurs marchandes de l'équipe.
    """
    valeurs = [market_value_player(df_player, player_id) for player_id in liste_player_id]
    valeurs = [v for v in valeurs if v is not None]  # Exclure les valeurs None
    return sum(valeurs)

# Exemple d'utilisation
liste_player_id = [855134, 655006, 395237, 131225, 93720, 607223, 955070, 711544, 37838, 60561, 187718] 
moyenne_valeur_marchande = mean_team_value(data_dict['players'], liste_player_id)
somme_valeur_marchande = sum_team_value(data_dict['players'], liste_player_id)

print("Valeur marchande moyenne de l'équipe :", moyenne_valeur_marchande)
print("Somme des valeurs marchandes de l'équipe :", somme_valeur_marchande)

Valeur marchande moyenne de l'équipe : 9090909.090909092
Somme des valeurs marchandes de l'équipe : 100000000.0


CREATION DU MODEL

In [336]:
#Configurations et imports
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [412]:
#pour cette première etape, on se base sur la forme de l'équipe et la valeur marchande des joueurs (indicatif de la qualité du joueur)
def calculer_forme(df_games, club_id, type):
    """Calcule la forme d'un club sur ses n derniers matchs avant la date du match."""
    # Filtrer les matchs précédents et trier par date
    matchs_precedents = games_of_the_season(df_games, club_id, 2022, type)
    
    # Calculer les points (exemple simplifié)
    points = 0
    for _, row in matchs_precedents.iterrows():
        if row['home_club_id'] == club_id:
            points += 3 if row['home_club_goals'] > row['away_club_goals'] else 1 if row['home_club_goals'] == row['away_club_goals'] else 0
        else:
            points += 3 if row['away_club_goals'] > row['home_club_goals'] else 1 if row['away_club_goals'] == row['home_club_goals'] else 0
    
    return points

In [430]:
#on travaille seulement sur la saison 2022 pour eviter les traitements inutiles
#matchs_2023 = data_dict["games"][data_dict["games"]['season'] == 2023]
# Supposons que 'data_dict["games"]' est votre DataFrame de matchs
df_games = data_dict["games"]

# Filtrer pour obtenir les matchs de la saison 2022 et des compétitions spécifiées
competitions_ids = ['FR1', 'ES1', 'IT1', 'L1', 'GB1'] #5 grands championnats
df_matchs = df_games[(df_games['season'] == 2022)]# & (df_games['competition_id'].isin(competitions_ids))]

df_matchs

,game_id,competition_id,season,round,date,home_club_id,away_club_id,home_club_goals,away_club_goals,home_club_position,...,stadium,attendance,referee,url,home_club_formation,away_club_formation,home_club_name,away_club_name,aggregate,competition_type
2644,3825606,DFB,2022,First Round,2022-08-01,4795,105,0,3,NaN,...,Audi Sportpark,5208.0,Robert Hartmann,https://www.transfermarkt.co.uk/spielbericht/i...,NaN,NaN,FC Ingolstadt 04,Sportverein Darmstadt 1898 e. V.,0:3,domestic_cup
2645,3828431,DK1,2022,4. Matchday,2022-08-05,369,865,3,3,10.0,...,Lyngby Stadion,5147.0,Aydin Uslu,https://www.transfermarkt.co.uk/lyngby-bk_fc-m...,NaN,NaN,Lyngby Boldklubben af 1921,Fodbold Club Midtjylland,3:3,domestic_league
2646,3828437,DK1,2022,5. Matchday,2022-08-15,678,369,1,0,3.0,...,Ceres Park,10646.0,Jörgen Daugbjerg Burchardt,https://www.transfermarkt.co.uk/aarhus-gf_lyng...,NaN,NaN,Aarhus Gymnastik Forening,Lyngby Boldklubben af 1921,1:0,domestic_league
2647,3828459,DK1,2022,8. Matchday,2022-09-04,865,1053,0,2,8.0,...,MCH Arena,8953.0,Morten Krogh,https://www.transfermarkt.co.uk/fc-midtjylland...,NaN,NaN,Fodbold Club Midtjylland,Aalborg BK,0:2,domestic_league
2648,3828486,DK1,2022,13. Matchday,2022-10-14,369,1053,0,2,12.0,...,Lyngby Stadion,4079.0,Jacob Karlsen,https://www.transfermarkt.co.uk/lyngby-bk_aalb...,NaN,NaN,Lyngby Boldklubben af 1921,Aalborg BK,0:2,domestic_league
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65813,3912539,EL,2022,Group A,2022-09-08,383,501,1,1,NaN,...,Philips Stadion,28627.0,Georgi Kabakov,https://www.transfermarkt.co.uk/spielbericht/i...,NaN,NaN,Eindhovense Voetbalvereniging Philips Sport Ve...,NaN,1:1,international_cup
65820,3872587,ECLQ,2022,Second Round 1st leg,2022-07-21,987,8780,0,1,NaN,...,Fir Park,7227.0,Kári Jóannesarson á Hövdanum,https://www.transfermarkt.co.uk/spielbericht/i...,NaN,NaN,Motherwell Football Club,NaN,0:1,international_cup
65831,3912540,EL,2022,Group A,2022-10-13,383,260,5,0,NaN,...,Philips Stadion,30000.0,Ali Palabiyik,https://www.transfermarkt.co.uk/spielbericht/i...,NaN,NaN,Eindhovense Voetbalvereniging Philips Sport Ve...,NaN,5:0,international_cup
65844,3865515,CIT,2022,First Round,2022-08-08,1025,4031,1,0,NaN,...,Stadio Renato Dall’Ara,10706.0,Giacomo Camplone,https://www.transfermarkt.co.uk/spielbericht/i...,NaN,NaN,Bologna Football Club 1909,NaN,1:0,domestic_cup


In [434]:
# Création des caractéristiques pour chaque match
features = []
labels = []
df_games = data_dict["games"]
df_matchs = df_matchs
df_game_lineup = data_dict["game_lineups"]
df_player_valuations = data_dict["players"]

for _, row in df_matchs.iterrows():
    home_forme = calculer_forme(df_matchs, row['home_club_id'], "domicile")
    away_forme = calculer_forme(df_matchs, row['away_club_id'], "exterieur")
    home_valeur_marchande = mean_team_value(df_player_valuations, starting_lineup_match(row["game_id"], row['home_club_id']))
    away_valeur_marchande = mean_team_value(df_player_valuations, starting_lineup_match(row["game_id"], row['away_club_id'])) 
    
    features.append([home_forme, away_forme, home_valeur_marchande, away_valeur_marchande])
    
    # Créer le label
    if row['home_club_goals'] > row['away_club_goals']:
        labels.append(1)
    elif row['home_club_goals'] < row['away_club_goals']:
        labels.append(-1)
    else:
        labels.append(0)

# Convertir en DataFrame pour l'utilisation avec des modèles de machine learning
df_features = pd.DataFrame(features, columns=['home_forme', 'away_forme', 'home_valeur_marchande', 'away_valeur_marchande'])
df_labels = pd.Series(labels)

In [435]:
print(df_labels)
print(df_features)

0      -1
1       0
2       1
3      -1
4      -1
       ..
5820    0
5821   -1
5822    1
5823    1
5824    1
Length: 5825, dtype: int64
      home_forme  away_forme  home_valeur_marchande  away_valeur_marchande
0              0           6                      0                      0
1              8          27                      0                      0
2             18           8                      0                      0
3             17          14                      0                      0
4              8          14                      0                      0
...          ...         ...                    ...                    ...
5820          63           2                      0                      0
5821          15          13                      0                      0
5822          63           6                      0                      0
5823          36           0                      0                      0
5824           6           9          

In [436]:
# Supprimer les lignes contenant des valeurs NaN
df_features_clean = df_features.dropna()
df_labels_clean = df_labels[df_features_clean.index]
df_features_clean
df_labels_clean

0      -1
1       0
2       1
3      -1
4      -1
       ..
5820    0
5821   -1
5822    1
5823    1
5824    1
Length: 5825, dtype: int64

EN UTILISANT UNE REGRESSION LOGISTIQUE

In [437]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Division des données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(df_features_clean, df_labels_clean, test_size=0.1, random_state=42)

# Création et entraînement du modèle
model = LogisticRegression()
model.fit(X_train, y_train)

# Évaluation du modèle
predictions = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, predictions))

Accuracy: 0.6329331046312179


In [438]:
def calculer_probabilites_victoire(df_games, df_players, model, liste_matchs):
    """
    Calcule les probabilités de victoire pour une liste de matchs.
    
    Paramètres:
    df_games (DataFrame) : DataFrame des matchs.
    df_players (DataFrame) : DataFrame des joueurs et de leurs valeurs marchandes.
    model (Model) : Modèle de machine learning entraîné.
    liste_matchs (List[Tuple[int, int]]) : Liste de tuples (home_club_id, away_club_id).
    
    Retourne:
    DataFrame : Probabilités de victoire pour chaque match.
    """
    # Liste pour stocker les caractéristiques de chaque match
    features_list = []
    
    # Préparer les caractéristiques pour chaque match
    for home_club_id, away_club_id in liste_matchs:
        home_forme = calculer_forme(df_games, home_club_id, "domicile")
        away_forme = calculer_forme(df_games, away_club_id, "exterieur")
        home_valeur_marchande = mean_team_value(df_players, starting_lineup_match(last_game_id(df_games, home_club_id), home_club_id))
        away_valeur_marchande = mean_team_value(df_players, starting_lineup_match(last_game_id(df_games, away_club_id), away_club_id))
        
        # Ajouter les caractéristiques pour le match courant à la liste
        features_list.append([home_club_id, away_club_id, home_forme, away_forme, home_valeur_marchande, away_valeur_marchande])
    
    # Créer un DataFrame avec toutes les caractéristiques
    df_features = pd.DataFrame(features_list, columns=['home_club_id', 'away_club_id', 'home_forme', 'away_forme', 'home_valeur_marchande', 'away_valeur_marchande'])
    
   # Préparer les caractéristiques pour la prédiction
    match_features = df_features[['home_forme', 'away_forme', 'home_valeur_marchande', 'away_valeur_marchande']]
    
    # Prédiction des probabilités avec le modèle pour chaque match
    probas = model.predict_proba(match_features)
    
    # Ajouter toutes les probabilités prédites au DataFrame
    for i, class_label in enumerate(model.classes_):
        df_features[f'prob_{class_label}'] = probas[:, i]
    
    # Sélectionner uniquement les colonnes 'home_club_id', 'away_club_id', et celles qui commencent par 'prob_'
    df_probabilites = df_features.filter(regex='^(home_club_id|away_club_id|prob_)')
    
    return df_probabilites

# Liste des matchs sous forme de tuples (home_club_id, away_club_id)
liste_matchs = [(1041, 583), (1041, 3524), (583, 3524), (1041, 162), (162, 1041), (418, 3302), (418, 12321), (31, 281), (281, 31), (418, 281), (281, 418)]  # Ajoutez d'autres matchs comme nécessaire

# Appel de la fonction et impression des résultats
df_probabilites = calculer_probabilites_victoire(df_games, df_players, model, liste_matchs)
print(df_probabilites)

    home_club_id  away_club_id   prob_-1    prob_0    prob_1
0           1041           583  0.642477  0.243328  0.114195
1           1041          3524  0.294800  0.262885  0.442315
2            583          3524  0.105066  0.223019  0.671915
3           1041           162  0.509424  0.273877  0.216699
4            162          1041  0.234869  0.252321  0.512810
5            418          3302  0.002903  0.043550  0.953547
6            418         12321  0.008074  0.078952  0.912974
7             31           281  0.241084  0.338141  0.420775
8            281            31  0.006593  0.092520  0.900887
9            418           281  0.091264  0.288621  0.620116
10           281           418  0.050385  0.277803  0.671811


In [410]:
"""
1041 = OL
583 = PSG
3524 = Clermont
162 = Monaco
418 = Real Madrid
3302 = Almeria
12321 = Girona
31 = Liverpool
281 = ManchesterCity
"""

'\n1041 = OL\n583 = PSG\n3524 = Clermont\n162 = Monaco\n418 = Real Madrid\n3302 = Almeria\n12321 = Girona\n31 = Liverpool\n281 = ManchesterCity\n'